<a href="https://colab.research.google.com/github/acoiman/pdt/blob/main/asthma_mortality/notebooks/Python/10.Asthma_Mortality_Panel_Data_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#🔮 Predicting Asthma Mortality Rate with Panel Data Models

Panel data models are defined as statistical models that utilize data containing observations over time for multiple entities. These models can incorporate fixed and random effects approaches to analyze the relationships between variables.


In this notebook explores the use of panel data  models to predict the Normalized Asthma Mortality Rate (NAMR) across departments in Argentina. By leveraging historical time series data from 2001 to 2021 and incorporating relevant environmental and demographic covariates, the objective is to estimate asthma mortality rate for 2022 and evaluate  performance of modfels using multiple error metrics.

##📦 Import and install required libraries

In [ ]:
# dataframe libraries
import pandas as pd
import numpy as np

# geodataframe libraries
import geopandas as gpd
import mapclassify

# plot libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from matplotlib.patches import Patch

# Modelling libraries
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects
from statsmodels.genmod.generalized_estimating_equations import GEE
from statsmodels.genmod.families import Binomial
from statsmodels.genmod.cov_struct import Exchangeable, Independence, Autoregressive, CovStruct, GlobalOddsRatio, Nested
import statsmodels.api as sm
from scipy import stats

# sklearn
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# other libraries
import os
import warnings
from itables import init_notebook_mode, show

In [ ]:
# Set the PROJ_LIB path
os.environ['PROJ_LIB'] = "/opt/conda/envs/gds/share/proj"

In [ ]:
 # Copy-on-Write
 pd.options.mode.copy_on_write = True

In [ ]:
# change to my computer home directory
%cd work

##🗂 Load and transform the dataset

In [ ]:
# Load dataset with data per department
gdf = gpd.read_file("pdt/asthma_mortality/data/gpkg/data.gpkg")

In [ ]:
# Reshape df to ts long format
years = range(2001, 2023)
records = []

In [ ]:
# drop geometry and PDPM25
for _, row in gdf.iterrows():
    iddpto = row["IDDPTO"]
    for year in years:
        records.append({
            "IDDPTO": iddpto,
            "YEAR": year,
            "CA": row.get(f"CA_{year}", np.nan),
            "PM25": row.get(f"PM25_{year}", np.nan),
            "NBA": row.get(f"NBA_{year}", np.nan),
            "PD": row.get(f"PD_{year}", np.nan),
            #"PDPM25": row.get(f"PDPM25_{year}", np.nan),
            "NAGRT": row.get(f"NAGRT_{year}", np.nan),
            "NNWVT": row.get(f"NNWVT_{year}", np.nan),
            "NBUT": row.get(f"NBUT_{year}", np.nan),
            "ELEV": row.get(f"ELEV_{year}", np.nan)
            })

In [ ]:
# create new df from list
df_ts = pd.DataFrame(records)

In [ ]:
# Create lag variables (up to 2 years)
def create_lags(df, var, max_lag=2):
    for lag in range(1, max_lag+1):
        df[f"{var}_lag{lag}"] = df[var].shift(lag)
    return df

In [ ]:
for var in ["NAGRT", "NNWVT", "NBUT"]:
    df_ts = create_lags(df_ts, var)

In [ ]:
# Drop the initial rows with NaNs due to lagging
df_ts = df_ts.dropna().reset_index(drop=True)

In [ ]:
# visusalize the dataframe
init_notebook_mode(all_interactive=True)
show(df_ts)

## 🧪 Apply the Hausman Test

We will apply the Hausman test to determine the appropriate choice between Fixed Effects (FE) and Random Effects (RE) models.

In [ ]:
# Ensure data panel index
df = df_ts.set_index(["IDDPTO", "YEAR"]).sort_index()

In [ ]:
# Train/test data split
train = df.loc[pd.IndexSlice[:, 2001:2021], :]
test  = df.loc[pd.IndexSlice[:, 2022], :]

In [ ]:
# Define features
features = [
    "PM25", "NBA", "PD",
    "NAGRT", "NNWVT", "NBUT",
    "NAGRT_lag1", "NAGRT_lag2",
    "NNWVT_lag1", "NNWVT_lag2",
    "NBUT_lag1", "NBUT_lag2",
    "ELEV"
]

In [ ]:
y = train['CA'] #
X = train[features]  # predictor variables
X = X.assign(const=1)  # RE and FE models needs constant

We wiil estimate a Fixed Effects (FE) Model to perform the Hausman Test and decide if FE is preferred that Random Effect (RE) model using the PaneLOS library.

PanelOLS is a statistical method used for analyzing panel data, which combines time-series and cross-sectional data . It is a form of fixed effects estimation that accounts for unobserved, entity-specific or time-specific characteristics, preventing biased results that might arise from ignoring these factors.

For PanelOLS we will set the following parameters:

* y: target variable
* X: predictor variables
* entity_effects=True: this specifies that we want to include fixed effects for each entity (department)
* drop_absorbed=True: this instructs the model to automatically remove any predictors that are perfectly collinear with the fixed effects.

In [ ]:
# Estimate Fixed Effects (FE) Model
model_fe = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
results_fe = model_fe.fit()

We will train a Random Effect modelv to perform the Hausman Test and decide if FE is preferred that Random Effect (RE) model using the RandomEffects library.

A random effects model is a statistical tool used to analyze data where observations are not independent, such as panel data, by accounting for unobserved differences between groups or individuals

In [ ]:
# Estimate Random Effects (RE) Model
model_re = RandomEffects(y, X)
results_re = model_re.fit()

In [ ]:
def hausman(fe, re):
    """
    fe, re: linearmodels results objects
    returns: test statistic, p-value
    """

    # Align coefficients (drop fixed-effects dummies)
    b_fe = fe.params
    b_re = re.params

    # Match variable names
    common_idx = b_fe.index.intersection(b_re.index)
    b_fe = b_fe[common_idx]
    b_re = b_re[common_idx]

    # Variances
    v_fe = fe.cov.loc[common_idx, common_idx]
    v_re = re.cov.loc[common_idx, common_idx]

    # Difference
    diff = b_fe - b_re
    v_diff = v_fe - v_re

    # Invert matrix
    try:
        inv_vdiff = np.linalg.inv(v_diff)
    except np.linalg.LinAlgError:
        raise ValueError("Variance matrix difference is singular. "
                         "Model may have multicollinearity.")

    # Hausman statistic
    H = float(diff.T @ inv_vdiff @ diff)

    df = len(diff)
    p_value = 1 - stats.chi2.cdf(H, df)

    return H, p_value

In [ ]:
# Run the Hausman test
H_stat, p_val = hausman(results_fe, results_re)

print("HAUSMAN TEST")
print("Test statistic:", H_stat)
print("Degrees of freedom:", len(results_fe.params))
print("p-value:", p_val)

if p_val < 0.05:
    print("\n👉 Reject H0: FE is preferred (RE inconsistent).")
else:
    print("\n👉 Fail to reject H0: RE is preferred (FE inefficient).")

##📝 Modelling Panel Data with Fixed Effects Model

In [ ]:
# Select test and tran dataset
y_train = train["CA"]

# predictors
predictors = [
    "PM25", "NBA", "PD",
    "NAGRT", "NNWVT", "NBUT",
    "NAGRT_lag1", "NAGRT_lag2",
    "NNWVT_lag1", "NNWVT_lag2",
    "NBUT_lag1", "NBUT_lag2",
    "ELEV"
]

X_train = train[predictors]
X_test  = test[predictors]

# Add constant
X_train = sm.add_constant(X_train)
X_test  = sm.add_constant(X_test)

#### Fit PanelOLS model

In this implementation of PanelOLS (Panel Ordinary Least Squares)  we especify that covariance estimator type is clustered using department  departments as entity clusters.

In [ ]:
model = PanelOLS(y_train, X_train, entity_effects=True, drop_absorbed=True)
results = model.fit(cov_type="clustered", cluster_entity=True)

#### Predicting NAMR (CA) for 2022

In [ ]:
# Ensure X_test has the same columns as the trained model (excluding 'ELEV')
X_test_pred = X_test.drop(columns=['ELEV'])

In [ ]:
# Predict for 2022
test["CA_2022_pred"]= results.predict(X_test_pred)

#### Evaluating prediction for 2022

In [ ]:
# Evaluate  prediction
mae   = mean_absolute_error(test["CA"], test["CA_2022_pred"])
rmse  = mean_squared_error(test["CA"], test["CA_2022_pred"], squared=False)

print("\nPerformance on 2022:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")

The model shows moderate predictive error for 2022.
An MAE of 1.72 indicates that predictions deviate on average by about 1–2 deaths per department, which may be substantial given the low counts and zero inflation in asthma mortality.
The RMSE of 2.38, higher than the MAE, suggests the presence of larger individual errors and that the model struggles particularly in departments with higher mortality values.

In [ ]:
# create df of results
df_results = test[["CA", "CA_2022_pred"]].rename(columns= {"CA": "CA_2022"})

In [ ]:
# create a scatterplot between CA_2022 vs forecast_2022
plt.figure(figsize=(10, 6))
plt.scatter(df_results['CA_2022'], df_results['CA_2022_pred'])
plt.xlabel('CA_2022 (Actual)')
plt.ylabel('forecast_2022 (Predicted)')
plt.title('Scatter Plot of Actual vs. Predicted Mortality Rates (2022)')
plt.grid(True)
plt.show()

The scatter plot shows systematic underperformance of the model. Most predictions cluster between 0 and 2, while actual mortality spans a much wider range—especially for departments with higher observed values, which are consistently under-predicted. The presence of several large negative predictions indicates clear model misspecification, likely due to zero inflation,or  non-normal residuals.

##  🚧 Hurdle-type Panel Data modelling

Hurdle-type modeling is a statistical approach designed to analyze count data, particularly when there is an excess of zeros. This model operates in two stages: the first stage determines whether the count is zero or positive using a binary classification, while the second stage models only the positive counts, often employing a regression models.[3]

We will apply this approach in two parts:

* Part 1: An GEE (Generalized Estimating Equations) Binomial model to predict the nonoccurrence (0) or occurrence (1) of asthma mortality (binary classification).

* Part 2:  A PanelOLS (Panel Ordinary Least Squares) model from 2001 to 2021 to predict NAMR values in 2022 where it is present (CA_bin == 1 or NAMR values > 0).

###  Binary model (GEE, Binomial)

In [ ]:
# copy df_ts
df = df_ts.copy()

In [ ]:
# Basic cleaning to ensure data types
df["YEAR"] = df["YEAR"].astype(int)
df["IDDPTO"] = df["IDDPTO"].astype(str)

In [ ]:
# Train/test split
# set panel index (useful for RandomEffects later)
df_panel = df.set_index(["IDDPTO", "YEAR"]).sort_index()

In [ ]:
df_train = df_panel.loc[pd.IndexSlice[:, 2001:2021], :].reset_index()
df_test  = df_panel.loc[pd.IndexSlice[:, 2022], :].reset_index()

In [ ]:
# Create binary outcome for the hurdle: 1 if CA > 0
df_train["y_bin"] = (df_train["CA"] > 0).astype(int)
df_test["y_bin"]  = (df_test["CA"] > 0).astype(int)

In [ ]:
init_notebook_mode(all_interactive=True)
df_train

In [ ]:
# choose predictors for the binary model
binary_features = [
    "PM25", "NBA", "PD",
    "NAGRT", "NNWVT", "NBUT",
    "NAGRT_lag1", "NAGRT_lag2",
    "NNWVT_lag1", "NNWVT_lag2",
    "NBUT_lag1", "NBUT_lag2",
    "ELEV"
]

In [ ]:
binary_features = [c for c in binary_features if c in df_train.columns]

We will train a GEE model, where each department (IDDPTO) is a cluster, meaning that  all observations belonging to the same department are correlated

* $family=Binomial()$ indicates that the response is binary (0/1).
* ${cov-struct=Autoregressive(grid=True)}$ indicates that observations closer in time are more correlated.
* $grid=True$ tells GEE to compute correlations using observed time spacing rather than assuming evenly spaced observations.

As the main assumption of the GEE model is that we must correctly describe how the predictors affect the mean (expected value) of the outcome.

In our case we tell the GEE model:

* which variables should predict the outcome → inside the formula

* how to convert that prediction into a probability → by choosing the Binomial family (logit link)

In [ ]:
# Build formula for GEE (statsmodels formula interface)
formula_bin = "y_bin ~ " + " + ".join(binary_features)

In [ ]:
# Fit GEE (Binomial) using an Autoregressive correlation within IDDPTO
gee_model = GEE.from_formula(formula_bin,
                             groups="IDDPTO",
                             data=df_train,
                             family=Binomial(),
                             cov_struct=Autoregressive(grid=True))

In [ ]:
gee_res = gee_model.fit(maxiter=1000)

In [ ]:
# model summary
gee_res.summary()

In [ ]:
gee_res.

#### Predicting the nonoccurrence (0) or occurrence (1) of asthma mortality  in 2022



In [ ]:
# predict uses exog from test data; we must pass test df to predict method
p_test = gee_res.predict(df_test)

In [ ]:
# ensure probabilities are in [0,1]
#p_test = np.clip(p_test, 0.0, 1.0)
df_test["p_pos"] = p_test

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
p_test

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
df_test

In [ ]:
# add IDDPTO and CA_bin (true labels for 2022)
ca_pred_2022 = df_test[['IDDPTO', "y_bin", "p_pos"]]

In [ ]:
# rename CA_bin to CA_2022_bin
ca_pred_2022 = ca_pred_2022.rename(columns={'y_bin': 'CA_2022_bin'})

In [ ]:
# visualize predictions vs true labels
init_notebook_mode(all_interactive=True)
ca_pred_2022

In [ ]:
# Apply a threshold (commonly 0.5) to the predicted probabilities to classify each observation as 0 or 1.
ca_pred_2022['CA_2022_bin_pred'] = (ca_pred_2022['p_pos'] >= 0.5).astype(int)

In [ ]:
# calculate RMMSE and MAE
rmse = np.sqrt(mean_squared_error(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred']))
mae = mean_absolute_error(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred'])

print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")

In [ ]:
# Calculate classification metrics, accuracy, precision, recall and f1_score
accuracy = accuracy_score(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred'])
precision = precision_score(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred'])
recall = recall_score(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred'])
f1 = f1_score(ca_pred_2022['CA_2022_bin'], ca_pred_2022['CA_2022_bin_pred'])

# Display the metrics
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")

The GEE model with AR(1) correlation shows good overall discrimination but poor sensitivity to positive asthma-mortality cases.

Accuracy (0.72) and precision (0.78) indicate that the model correctly identifies most  departments with nonoccurrence (0) of asthma mortality and produces few false positives.

However, recall is very low (0.23), meaning the model fails to detect most departments with occurence(1)  of asthma mortality, which leads to a modest F1-score (0.35).

The low RMSE (0.53) and MAE (0.28) suggest that predicted probabilities are numerically close to true values, but the classification thresholding strongly favors zeros, which is typical with rare and zero-inflated outcomes.

Overall, the model captures general patterns but is not effective at identifying actual oocurence of mortality events.


###  Regression model (PanelOLS)

In this section, we will train and evaluate a PanelOLS (Panel Ordinary Least Squares) model from 2001 to 2021 to predict NAMR values in 2022 where it is present (CA_bin == 1 or NAMR values > 0).

#### Hausman Test

We will apply the Hausman test to determine the appropriate choice between Fixed Effects (FE) and Random Effects (RE) models.

In [ ]:
# Filter df_train to keep only rows where CA_bin is 1
train_pos = df_train[(df_train['y_bin'] == 1)]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
train_pos

In [ ]:
# Ensure data panel index
train_pos = train_pos.set_index(["IDDPTO", "YEAR"]).sort_index()

In [ ]:
# drop y_bin column
train_pos = train_pos.drop(columns=['y_bin'])

In [ ]:
init_notebook_mode(all_interactive=True)
train_pos

In [ ]:
# Define features
features = [
    "PM25", "NBA", "PD",
    "NAGRT", "NNWVT", "NBUT",
    "NAGRT_lag1", "NAGRT_lag2",
    "NNWVT_lag1", "NNWVT_lag2",
    "NBUT_lag1", "NBUT_lag2",
    "ELEV"
]

In [ ]:
y = train_pos['CA'] # taget variable
X = train_pos[features]  # predictor variables
X = X.assign(const=1)  # RE and FE models needs constant

In [ ]:
# Estimate Fixed Effects (FE) Model
model_fe = PanelOLS(y, X, entity_effects=True, drop_absorbed=True)
results_fe = model_fe.fit()

In [ ]:
# Estimate Random Effects (RE) Model
model_re = RandomEffects(y, X)
results_re = model_re.fit()

In [ ]:
def hausman(fe, re):
    """
    fe, re: linearmodels results objects
    returns: test statistic, p-value
    """

    # Align coefficients (drop fixed-effects dummies)
    b_fe = fe.params
    b_re = re.params

    # Match variable names
    common_idx = b_fe.index.intersection(b_re.index)
    b_fe = b_fe[common_idx]
    b_re = b_re[common_idx]

    # Variances
    v_fe = fe.cov.loc[common_idx, common_idx]
    v_re = re.cov.loc[common_idx, common_idx]

    # Difference
    diff = b_fe - b_re
    v_diff = v_fe - v_re

    # Invert matrix
    try:
        inv_vdiff = np.linalg.inv(v_diff)
    except np.linalg.LinAlgError:
        raise ValueError("Variance matrix difference is singular. "
                         "Model may have multicollinearity.")

    # Hausman statistic
    H = float(diff.T @ inv_vdiff @ diff)

    df = len(diff)
    p_value = 1 - stats.chi2.cdf(H, df)

    return H, p_value

In [ ]:
# Run the Hausman test
H_stat, p_val = hausman(results_fe, results_re)

print("HAUSMAN TEST")
print("Test statistic:", H_stat)
print("Degrees of freedom:", len(results_fe.params))
print("p-value:", p_val)

if p_val < 0.05:
    print("\n👉 Reject H0: FE is preferred (RE inconsistent).")
else:
    print("\n👉 Fail to reject H0: RE is preferred (FE inefficient).")

#### Regression Model with Fixed Effects Model

In [ ]:
# Filter df_test to keep only rows where CA_bin is 1
test_pos = df_test[(df_test['y_bin'] == 1)]

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
test_pos

In [ ]:
# Ensure data panel index
test_pos = test_pos.set_index(["IDDPTO", "YEAR"]).sort_index()

In [ ]:
# drop y_bin column
test_pos = test_pos.drop(columns=['y_bin'])

In [ ]:
# visualizee dataframe
init_notebook_mode(all_interactive=True)
test_pos

In [ ]:
# Select test and tran dataset
y_train = train_pos["CA"]

# predictors
predictors = [
    "PM25", "NBA", "PD",
    "NAGRT", "NNWVT", "NBUT",
    "NAGRT_lag1", "NAGRT_lag2",
    "NNWVT_lag1", "NNWVT_lag2",
    "NBUT_lag1", "NBUT_lag2",
    "ELEV"
]

X_train = train_pos[predictors]
X_test  = test_pos[predictors]

# Add constant
X_train = sm.add_constant(X_train)
X_test  = sm.add_constant(X_test)

In [ ]:
# visualize dataframe
init_notebook_mode(all_interactive=True)
X_test

#### Fit PanelOLS model

The main assumption of the PanelOLS model are the following:

* Every department has unobserved characteristics that do not change over time. In our case elevation.
* These characteristics are correlated with your predictors. In our case this assumptiob is not true
* The fixed-effects transformation removes them.In our case it will be revoved.
* Remaining variation over time can be used to estimate the causal/associative effect of predictors.

For PanelOLS we will set the following parameters:

* y: target variable
* X: predictor variables
* entity_effects=True: this specifies that we want to include fixed effects for each entity (department)
* drop_absorbed=True: this instructs the model to automatically remove any predictors that are perfectly collinear with the fixed effects

In [ ]:
model = PanelOLS(y_train, X_train, entity_effects=True, drop_absorbed=True)
results = model.fit(cov_type="clustered", cluster_entity=True)

In [ ]:
# model summary
results

#### Predicting NAMR (CA) for 2022

In [ ]:
# Ensure X_test has the same columns as the trained model (excluding 'ELEV')
X_test_pred = X_test.drop(columns=['ELEV'])

In [ ]:
# Predict for 2022
test_pos["CA_2022_pred"]= results.predict(X_test_pred)

In [ ]:
init_notebook_mode(all_interactive=True)
test_pos

#### Evaluating prediction for 2022

In [ ]:
# Evaluate  prediction
mae   = mean_absolute_error(test_pos["CA"], test_pos["CA_2022_pred"])
rmse  = mean_squared_error(test_pos["CA"], test_pos["CA_2022_pred"], squared=False)

print("\nPerformance on 2022:")
print(f"MAE  = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")

In [ ]:
# calcualte r2 score
r2 = r2_score(test_pos["CA"], test_pos["CA_2022_pred"])
r2

In [ ]:
# create df of results
df_results = test_pos[["CA", "CA_2022_pred"]].rename(columns= {"CA": "CA_2022"})

In [ ]:
# create a scatterplot between CA_2022 vs forecast_2022
plt.figure(figsize=(10, 6))
plt.scatter(df_results['CA_2022'], df_results['CA_2022_pred'])
plt.xlabel('CA_2022 (Actual)')
plt.ylabel('forecast_2022 (Predicted)')
plt.title('Scatter Plot of Actual vs. Predicted Asthma Mortality Rates (2022)')
plt.grid(True)
plt.show()

The PanelOLS model shows very limited predictive ability for asthma mortality in 2022. The MAE (2.30) and RMSE (2.84) indicate substantial average prediction errors relative to the scale of the outcome, and the R² is slightly negative, meaning the model performs worse than simply predicting the mean. The scatter plot confirms this: predictions cluster around a narrow band and fail to follow the variability of the actual values—especially underestimating high rates and producing some unrealistic negative predictions. Overall, the model captures almost none of the true spatial variation in 2022 mortality.